In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


# load the music data into a pandas datafr
music_df = pd.read_csv("my dataset 6.csv")

music_df['genre'] = music_df['genre'].apply(lambda x: re.sub(r'[^\w\s]', '', x))  # Remove punctuation
music_df['genre'] = music_df['genre'].str.lower()  # Convert genre to lowercase
# Remove duplicate elements
music_df['genre'] = music_df['genre'].apply(lambda x: ' '.join(set(x.split())))

tfidf_vectorizer = TfidfVectorizer(stop_words='english')


# Fit the vectorizer to the lyrics in the dataset
tfidf_matrix = tfidf_vectorizer.fit_transform(music_df['genre'])


def get_similar_songs(song_title, n=10):
    # Get the index of the song with the given title
    idx = music_df[music_df['song name'] == song_title].index[0]

    # Compute the pairwise cosine similarity between the input song and all other songs in the dataset
    sim_scores = cosine_similarity(tfidf_matrix[idx], tfidf_matrix)

    # Get the indices of the top n most similar songs (excluding the input song)
    top_indices = sim_scores.argsort()[0][-n-1:-1][::-1]

    # Return the titles of the top n most similar songs
    return music_df['song name'].iloc[top_indices]

# user input
song=input("Enter a song:-")
similar_songs= get_similar_songs(song, n=5)
print(similar_songs)

# split the data into training and test sets
train_df = music_df[:1600]
test_df = music_df[1600:]

# create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# fit the vectorizer to the training data
tfidf_matrix_train = tfidf_vectorizer.fit_transform(train_df['genre'])

# evaluate the performance of the recommendation system using the test set
accuracy = 0
precision = 0

for song_title in test_df['song name']:
    # get the top most similar songs for the current song in the test set
    similar_songs = get_similar_songs(song_title, n=260)

    # check if the true song title is in the list of top 5 most similar songs
    if song_title in similar_songs.tolist():
        accuracy += 1
        precision += 1
       
    else:
        false_positives = [song for song in similar_songs.tolist() if song not in test_df['song name'].tolist()]
        false_negatives = [song for song in test_df['song name'].tolist() if song not in similar_songs.tolist()]
        precision += len([song for song in similar_songs.tolist() if song in test_df['song name'].tolist()]) / len(similar_songs)
        

accuracy /= len(test_df)
precision /= len(test_df)


print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")


Enter a song:-Sunno a buke
138    Aami Natun Kare Gorbo Thakur  
128            He Madhaba He Madhabo 
129         Shiuli Phuler Mala Dole  
130          Ebar Nabin Mantre Hobe  
131         Aslo Jakhon Phuler Fagun 
Name: song name, dtype: object
Accuracy: 0.73
Precision: 0.80
